In [37]:
import logging
from tqdm import tqdm
import numpy as np
import pandas as pd
from hnmchallenge.constant import *
from hnmchallenge.datasets.all_items_last_month_last_week import AILMLWDataset
from hnmchallenge.models.itemknn.itemknn import ItemKNN
from hnmchallenge.models_prediction.recs_interface import RecsInterface
from hnmchallenge.models_prediction.itemknn_recs import ItemKNNRecs
from hnmchallenge.utils.logger import set_color

In [26]:
map_score=[]
recall_score=[]

In [38]:
dataset=AILMLWDataset()
full_data = dataset.get_holdin()
t_dat = full_data["t_dat"].max()

In [40]:
t_dat1=t_dat - pd.to_timedelta(2+1, unit="D")

In [42]:
print(t_dat1)
cut=[12,50]#,100]#,200]

2020-09-12 00:00:00


In [31]:
dataset=AILMLWDataset()
map_score=[]
recall_score=[]
for j in cut:   
    x=[]
    y=[]
    for i in tqdm(range(2)):
        full_data = dataset.get_full_data()
        t_dat = full_data["t_dat"].max()
        t_dat1=t_dat - pd.to_timedelta(i+1, unit="D")
        candidate_items = full_data[full_data["t_dat"] >= t_dat1][["article_id"]].drop_duplicates()
        candidate_items = candidate_items.values.squeeze()
        TW = True
        REMOVE_SEEN = False
        FC = True
        dataset=AILMLWDataset()
        for kind in ["train"]:  # , "full"]:        # for kind in ["train", "full"]:
            rec_ens = ItemKNNRecs(
                kind=kind,
                cutoff=j,
                time_weight=TW,
                remove_seen=REMOVE_SEEN,
                dataset=dataset,
                filter_on_candidates=candidate_items,
            )
            score, recall=rec_ens.eval_recommendations()
            x.append(score)
            y.append(recall)
    map_score.append(x)
    recall_score.append(y)

  0%|          | 0/2 [00:00<?, ?it/s]

hnmchallenge/models_prediction/evaluation_logs/ItemKNN_tw_True_rs_False__17_07_2022__15_46_37.log
Computing similarity...


Done: 100%|██████████| 104547/104547 [00:03<00:00, 28211.72it/s]


  0%|          | 0/4 [00:01<?, ?it/s]

Filtering scores on candidates...
Filtering scores on candidates...
Filtering scores on candidates...
Filtering scores on candidates...
63412
63412


...

hnmchallenge/models_prediction/evaluation_logs/ItemKNN_tw_True_rs_False__17_07_2022__15_54_51.log
Computing similarity...


Done: 100%|██████████| 104547/104547 [00:03<00:00, 27330.12it/s]


  0%|          | 0/4 [00:01<?, ?it/s]

Filtering scores on candidates...
Filtering scores on candidates...
Filtering scores on candidates...
Filtering scores on candidates...
63412
63412


100%|██████████| 2/2 [05:37<00:00, 168.70s/it]


In [32]:
print(map_score)

[[0.02876718492647003, 0.028658134749787127], [0.030792360285050082, 0.03069527910866924]]


In [34]:
from pathlib import Path
import os

DATASET_NAME = "AILMLW_dataset"
_DATA_PATH = Path(Path.home() / os.environ.get("DATA_PATH"))
_DATASET_PATH = _DATA_PATH / "datasets" / DATASET_NAME
_HOLDIN_PATH = Path(_DATASET_PATH / Path("map_score.feather"))
df={'MAP@12':map_score[0], 'recall@12':recall_score[0], 'MAP@50':map_score[1], 'recall@50':recall_score[1] }
article_score=pd.DataFrame(df)
article_score.to_feather(_HOLDIN_PATH)

In [36]:
article_score

,MAP@12,recall@12,MAP@50,recall@50
0,0.028767,0.058788,0.030792,0.097785
1,0.028658,0.058517,0.030695,0.097736
